# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Initialize-Environment" data-toc-modified-id="Initialize-Environment-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Initialize Environment</a></div><div class="lev2 toc-item"><a href="#Generate-List-of-Data" data-toc-modified-id="Generate-List-of-Data-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Generate List of Data</a></div><div class="lev1 toc-item"><a href="#Modules-vs-Subgraphs" data-toc-modified-id="Modules-vs-Subgraphs-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Modules vs Subgraphs</a></div><div class="lev2 toc-item"><a href="#Map-Subgraph-Topology-to-Module-Allegiance-Matrix" data-toc-modified-id="Map-Subgraph-Topology-to-Module-Allegiance-Matrix-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Map Subgraph Topology to Module Allegiance Matrix</a></div><div class="lev3 toc-item"><a href="#Plot-Ranked-Module-Sensitivity" data-toc-modified-id="Plot-Ranked-Module-Sensitivity-211"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Plot Ranked Module Sensitivity</a></div><div class="lev3 toc-item"><a href="#Plot-Subgraphs" data-toc-modified-id="Plot-Subgraphs-212"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Plot Subgraphs</a></div><div class="lev2 toc-item"><a href="#Compute-Subgraph-and-Module-Dynamics" data-toc-modified-id="Compute-Subgraph-and-Module-Dynamics-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Compute Subgraph and Module Dynamics</a></div><div class="lev2 toc-item"><a href="#Temporal-Dynamics-of-Inter/Intra-Module-Processing" data-toc-modified-id="Temporal-Dynamics-of-Inter/Intra-Module-Processing-23"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Temporal Dynamics of Inter/Intra Module Processing</a></div><div class="lev3 toc-item"><a href="#Plot-subgraphs-with-high-and-low-coupling-strengths" data-toc-modified-id="Plot-subgraphs-with-high-and-low-coupling-strengths-231"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>Plot subgraphs with high and low coupling strengths</a></div><div class="lev3 toc-item"><a href="#Plot-the-average-temporal-correlation-of-subgraphs" data-toc-modified-id="Plot-the-average-temporal-correlation-of-subgraphs-232"><span class="toc-item-num">2.3.2&nbsp;&nbsp;</span>Plot the average temporal correlation of subgraphs</a></div><div class="lev3 toc-item"><a href="#Plot-the-temporal-correlation-between-subgraph-types" data-toc-modified-id="Plot-the-temporal-correlation-between-subgraph-types-233"><span class="toc-item-num">2.3.3&nbsp;&nbsp;</span>Plot the temporal correlation between subgraph types</a></div><div class="lev3 toc-item"><a href="#Module-Reorganization-via-Subgraph-Expression-Coupling" data-toc-modified-id="Module-Reorganization-via-Subgraph-Expression-Coupling-234"><span class="toc-item-num">2.3.4&nbsp;&nbsp;</span>Module Reorganization via Subgraph Expression Coupling</a></div><div class="lev2 toc-item"><a href="#Regional-variation-in-module-flexibility-and-subgraph-coupling" data-toc-modified-id="Regional-variation-in-module-flexibility-and-subgraph-coupling-24"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Regional variation in module flexibility and subgraph coupling</a></div><div class="lev3 toc-item"><a href="#Project-Module-based-node-flexibility-onto-Brain" data-toc-modified-id="Project-Module-based-node-flexibility-onto-Brain-241"><span class="toc-item-num">2.4.1&nbsp;&nbsp;</span>Project Module-based node flexibility onto Brain</a></div><div class="lev3 toc-item"><a href="#Project-Subgraph-based-node-coupling-onto-Brain" data-toc-modified-id="Project-Subgraph-based-node-coupling-onto-Brain-242"><span class="toc-item-num">2.4.2&nbsp;&nbsp;</span>Project Subgraph-based node coupling onto Brain</a></div>

# Initialize Environment

In [3]:
try:
    %load_ext autoreload
    %autoreload 2
    %reset -f
except:
    print 'NOT IPYTHON'

from __future__ import division

import os
import sys
import glob

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import scipy.io as io
import h5py
import matplotlib.pyplot as plt
from matplotlib import rcParams

sys.path.append('/Users/akhambhati/Developer/hoth_research/Echobase')
import Echobase
convert_conn_vec_to_adj_matr = Echobase.Network.Transforms.configuration.convert_conn_vec_to_adj_matr
convert_adj_matr_to_cfg_matr = Echobase.Network.Transforms.configuration.convert_adj_matr_to_cfg_matr

rcParams = Echobase.Plotting.fig_format.update_rcparams(rcParams)

path_AtlasData = '/Users/akhambhati/Remotes/CORE.MRI_Atlases'
path_CoreData = '/Users/akhambhati/Remotes/CORE.fMRI_multiband.mmattar/restdata'
path_PeriphData = '/Users/akhambhati/Remotes/RSRCH.NMF_Subnetworks'
path_ModData = path_PeriphData + '/e02c-DynFuncModule-Population'
path_SubData = path_PeriphData + '/e03c-DynFuncSubgraph-Population'
path_ExpData = path_PeriphData + '/e04c-CmpModuleSubgraph-Population'

for path in [path_CoreData, path_PeriphData, path_ModData, path_SubData, path_ExpData]:
    if not os.path.exists(path):
        print('Path: {}, does not exist'.format(path))
        os.makedirs(path)     
        
if not os.path.exists('./e04c-Figures'):
    os.makedirs('./e04c-Figures')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Generate List of Data

In [4]:
data_dict = {'module_path': glob.glob('{}/Module_Optimization.ModAssign.*.mat'.format(path_ModData)),
             'module_alleg_path': '{}/consensus_module_allegiance.npz'.format(path_ModData),
             'subgraph_path': '{}/NMF_Optimization.consensus_subgraph.npz'.format(path_SubData),
             'cfg_path': '{}/NMF_Optimization.CfgMatr.npz'.format(path_SubData),
             'subgraph_edge_null_path': glob.glob('{}/NMF_EdgeNull.subgraph_seed-*.npz'.format(path_SubData))}

# Modules vs Subgraphs

## Map Subgraph Topology to Module Allegiance Matrix

In [ ]:
# Load Module Allegiance data
data_mda = np.load(data_dict['module_alleg_path'], mmap_mode='r')
mda_vec = convert_adj_matr_to_cfg_matr(np.expand_dims(data_mda['module_alleg'], axis=0)).reshape(-1)

# Load Subgraph Data
data_sub = np.load(data_dict['subgraph_path'], mmap_mode='r')
fac_subgraph = data_sub['fac_subnet']

# Get Data Params
n_fac = fac_subgraph.shape[0]
n_conn = fac_subgraph.shape[1]
n_null = len(data_dict['subgraph_edge_null_path'])

# Initialize buckets
msi_real = np.nan*np.zeros(n_fac)
msi_edge_null = np.nan*np.zeros((n_fac, n_null))

# Correlation between Subgraph and Distance Matrix
for f_id in xrange(n_fac):
    rho, pval = stats.pearsonr(mda_vec, fac_subgraph[f_id, :])
    msi_real[f_id] = rho
    
# Edge Null Correlation between Subgraph and Distance Matrix
for n_i, null_path in enumerate(data_dict['subgraph_edge_null_path']):
    data_null_sub = np.load(null_path, mmap_mode='r')
    fac_subgraph = data_null_sub['fac_subnet']

    for f_id in xrange(n_fac):
        rho, pval = stats.pearsonr(mda_vec, fac_subgraph[f_id, :])
        msi_edge_null[f_id, n_i] = rho

np.savez('{}/Module_Sensitivity.npz'.format(path_ExpData),
         msi_real=msi_real,
         msi_edge_null=msi_edge_null)

### Plot Ranked Module Sensitivity
$\text{Subgraph}_{TRUE}$ -- represents groups of temporally co-varying edge strengths with in-tact geometry and topology -- Topographic sensitivity of subgraphs driven by temporal structure of geometry and topology

$\text{Subgraph}_{RW}$ -- represents the ``noise floor`` of the temporal co-variance without geometry or topology -- Topographic sensitivity of subgraphs driven purely by temporal structure

$\text{Subgraph}_{GRW}$ -- represents the ``noise floor`` of the temporal co-variance with in-tact geometry, but no topology -- Topographic sensitivity of subgraphs driven purely by temporal structure of geometry, but no topology

In [ ]:
# Load Data
df = np.load('{}/Module_Sensitivity.npz'.format(path_ExpData))

msi_real = df['msi_real']
msi_edge_null = df['msi_edge_null']
msi_ord_ix = np.argsort(msi_real)

n_fac, n_null = msi_edge_null.shape

### Plot Module Sensitivity Index
plt.figure(figsize=(3,3))
ax = plt.subplot(111)

# Real Topography
ax.plot(np.arange(n_fac)+1, np.sort(msi_real),
        color=[0.2, 0.2, 0.2])

alpha = 0.025
subg_intra_mod = []
subg_inter_mod = []
for ix, msi in enumerate(np.sort(msi_real)):
    if np.mean(msi_edge_null.reshape(-1) > msi) < alpha:
        color=[0.75, 0.20, 0.20]
        subg_intra_mod.append(ix)
    elif np.mean(msi_edge_null.reshape(-1) < msi) < alpha:
        color=[0.20, 0.20, 0.75] 
        subg_inter_mod.append(ix)
    else:
        color=[0.20, 0.20, 0.20]
        
    ax.scatter(ix+1, msi,
               lw=0, color=color, alpha=1.0, s=40.0)

# Geometric and Edge Null Topography
ax.fill_between([1, n_fac],
                y1=np.percentile(msi_edge_null.reshape(-1), alpha*100),
                y2=np.percentile(msi_edge_null.reshape(-1), (1-alpha)*100),
                color=[1.00, 0.25, 0.25], alpha=0.2, lw=0)

# Axis Settings
ax.set_xlim([0, n_fac+1])
#ax.set_ylim([-0.2, 0.6])
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
ax.set_ylabel('Pearson Correlation\n(Edge Strength vs Module Allegiance)');
ax.set_xlabel('Subgraphs');

plt.savefig('./e04c-Figures/Module_Sensitivity_Distribution.svg')
plt.show()
plt.close()

np.savez('{}/Module_Sensitivity.npz'.format(path_ExpData),
         msi_real=msi_real,
         msi_ord_ix=msi_ord_ix,
         msi_edge_null=msi_edge_null,
         subg_intra_mod = subg_intra_mod,
         subg_inter_mod = subg_inter_mod)

### Plot Subgraphs

In [ ]:
# Load Subgraph Data
data_sub = np.load(data_dict['subgraph_path'], mmap_mode='r')
fac_subgraph = data_sub['fac_subnet']

# Load Module Sensitivity Index
df = np.load('{}/Module_Sensitivity.npz'.format(path_ExpData))
msi_ord_ix = df['msi_ord_ix']

sel_fac = [('InterM', msi_ord_ix[0]),
           ('IntraM', msi_ord_ix[-1])]

# Read the ROI CSV
roi_csv = pd.read_csv('{}/fsl/HarvardOxford/HOA112_Labels.csv'.format(path_AtlasData))
roi_coords = np.vstack((roi_csv['X'], roi_csv['Y'], roi_csv['Z'])).T

view_angle = {'Axial_LR': [0.0, 0.0],
              'Axial_RL': [0.0, 180.0],
              'Sag_PA': [0.0, 90.0],
              'Sag_AP': [180.0, 90.0]}

for subg_type, ix in sel_fac:
    Echobase.Plotting.render_brain_connectivity.draw(vtk_files=['{}/fsl/MNI/rh.pial.vtk'.format(path_AtlasData),
                                                                '{}/fsl/MNI/lh.pial.vtk'.format(path_AtlasData)],
                                                     brain_rgba=(0.5, 0.5, 0.5, 0.05), conn_cmap='plasma',
                                                     node_coords=roi_coords, conn_thr=[0.975, 1.0],
                                                     conn_list=fac_subgraph[ix, :] / fac_subgraph[ix, :].max())
    for ang in view_angle.keys():
        Echobase.Plotting.render_brain_connectivity.mlab.view(azimuth=view_angle[ang][0],
                                                              elevation=view_angle[ang][1])
        Echobase.Plotting.render_brain_connectivity.mlab.savefig('./e04c-Figures/Subgraph-{}-{}.png'.format(subg_type, ang))
    Echobase.Plotting.render_brain_connectivity.mlab.close(all=True)

## Compute Subgraph and Module Dynamics

In [ ]:
# Load Module Sensitivity Index
df = np.load('{}/Module_Sensitivity.npz'.format(path_ExpData))
msi_ord_ix = df['msi_ord_ix']

# Load cfg data
data_cfg = np.load(data_dict['cfg_path'], mmap_mode='r')
subj_coef_full_lbl = data_cfg['cfg_name']
subj_coef_lbl = np.array([name.split('.')[0] for name in data_cfg['cfg_name']])
subj_lbl = np.unique(subj_coef_lbl)
n_subj = len(subj_lbl)
n_win_per_subj = len(np.flatnonzero(subj_coef_lbl == subj_lbl[0]))
n_win_per_run = 20
n_run_per_subj = int(n_win_per_subj / n_win_per_run)

# Load Subgraph Data
data_sub = np.load(data_dict['subgraph_path'], mmap_mode='r')
fac_subgraph = data_sub['fac_subnet']
fac_coef = data_sub['fac_coef'][msi_ord_ix, :]    # **Order by MSI index

# Get Data Params
n_fac = fac_subgraph.shape[0]
n_conn = fac_subgraph.shape[1]
n_node = np.int(np.ceil(np.sqrt(n_conn*2)))
n_mod_seed = len(data_dict['module_path'])

# Buckets for Subgraph dynamics
subgraph_expr_corr = np.nan*np.zeros((n_fac, n_fac, n_subj, n_run_per_subj))

# Compute Subgraph Expression Dynamics
for s_id, subj_id in enumerate(subj_lbl):
    win_ix = np.flatnonzero(subj_coef_lbl == subj_id)
    
    run_lbl = np.unique(subj_coef_full_lbl[win_ix])
    for r_id, r_lbl in enumerate(run_lbl):
        run_ix = np.flatnonzero(subj_coef_full_lbl == r_lbl)

        sel_fac_coef = fac_coef[:, run_ix]
        subgraph_expr_corr[:, :, s_id, r_id] = np.corrcoef(sel_fac_coef**2)

# Compute module flexibility
module_flex = np.nan*np.zeros((n_subj, n_run_per_subj, n_node, n_mod_seed))
for mseed_id, mseed_path in enumerate(data_dict['module_path']):
    data_mseed = io.loadmat(mseed_path)
    for s_id in xrange(n_subj):
        module_assign = np.array(data_mseed['Ssubj'][0, s_id], float).T
        
        for r_id in xrange(n_run_per_subj):
            module_run = module_assign[r_id*n_win_per_run:(r_id+1)*n_win_per_run, :]
            module_flex[s_id, r_id, :, mseed_id] = np.nanmean(np.abs(np.diff(module_run, axis=0)) > 0, axis=0)                
                
np.savez('{}/Subgraph_Module_Dynamics.npz'.format(path_ExpData),
         module_flex = module_flex,
         subgraph_expr_corr = subgraph_expr_corr)

## Temporal Dynamics of Inter/Intra Module Processing

In [ ]:
# Load Temporal Expression Correlation
df = np.load('{}/Subgraph_Module_Dynamics.npz'.format(path_ExpData), mmap_mode='r')
subg_expr_corr = df['subgraph_expr_corr']
n_fac, n_fac, n_subj, n_run = subg_expr_corr.shape

# Load Within-Module and Between-Module Subgraphs
df = np.load('{}/Module_Sensitivity.npz'.format(path_ExpData))
subg_intra_mod = df['subg_intra_mod']
subg_inter_mod = df['subg_inter_mod']

### Distribution of Temporal Correlation of Subgraphs
## Correlation among Inter-Module Subgraphs
triu_ix, triu_iy = np.triu_indices(len(subg_inter_mod), k=1)

# Real
expr_corr_inter_mod = subg_expr_corr[np.meshgrid(subg_inter_mod, subg_inter_mod)]
expr_corr_inter_mod_dist_lvoo = []
expr_corr_inter_mod_dist_lvoo.append(expr_corr_inter_mod[triu_ix, triu_iy, :, :].mean(axis=0).mean(axis=-1))
for r_id in xrange(n_run):
    lvin = np.setdiff1d(range(n_run), r_id)
    expr_corr_inter_mod_dist_lvoo.append(
        expr_corr_inter_mod[triu_ix, triu_iy, :, :][:, :, lvin].mean(axis=0).mean(axis=-1))

## Correlation among Intra-Module Subgraphs
triu_ix, triu_iy = np.triu_indices(len(subg_intra_mod), k=1)

# Real
expr_corr_intra_mod = subg_expr_corr[np.meshgrid(subg_intra_mod, subg_intra_mod)]
expr_corr_intra_mod_dist_lvoo = []
expr_corr_intra_mod_dist_lvoo.append(expr_corr_intra_mod[triu_ix, triu_iy, :, :].mean(axis=0).mean(axis=-1))
for r_id in xrange(n_run):
    lvin = np.setdiff1d(range(n_run), r_id)
    expr_corr_intra_mod_dist_lvoo.append(
        expr_corr_intra_mod[triu_ix, triu_iy, :, :][:, :, lvin].mean(axis=0).mean(axis=-1))

## Correlation between Intra-Module and Inter-Module Subgraphs
# Real
expr_corr_betwn_mod = subg_expr_corr[np.meshgrid(subg_inter_mod, subg_intra_mod)]
expr_corr_betwn_mod_dist_lvoo = []
expr_corr_betwn_mod_dist_lvoo.append(expr_corr_betwn_mod.mean(axis=0).mean(axis=0).mean(axis=-1))
for r_id in xrange(n_run):
    lvin = np.setdiff1d(range(n_run), r_id)
    expr_corr_betwn_mod_dist_lvoo.append(
        expr_corr_betwn_mod[:, :, :, lvin].mean(axis=0).mean(axis=0).mean(axis=-1))

### Plot subgraphs with high and low coupling strengths

In [ ]:
high_couple_ix = np.nonzero((subg_expr_corr.mean(axis=-1) < 0.98) &
                            (subg_expr_corr.mean(axis=-1) > 0.50))
low_couple_ix = np.nonzero((subg_expr_corr.mean(axis=-1) > -0.98) &
                           (subg_expr_corr.mean(axis=-1) < -0.50))

plt.figure()
ax = plt.subplot(111)
for cp_ix, (couple_name, couple_ix) in enumerate([('High_Couple', high_couple_ix),
                                                  ('Low_Couple', low_couple_ix)]):
    sg1 = couple_ix[0][0]
    sg2 = couple_ix[1][0]
    subj_ix = couple_ix[2][0]
    
    win_ix = np.flatnonzero(subj_coef_lbl == subj_lbl[subj_ix])
    sel_fac_coef = fac_coef[:, win_ix]
    
    fac1 = sel_fac_coef[sg1, :] / (1.25*sel_fac_coef[sg1, :].max())
    fac2 = sel_fac_coef[sg2, :] / (1.25*sel_fac_coef[sg2, :].max())
    
    ax.plot((fac1-fac1.mean())+cp_ix+1)
    ax.plot((fac2-fac2.mean())+cp_ix+1)

ax.set_ylim([0.5, 2.5])
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
ax.set_xlabel('Time')

plt.savefig('./e04c-Figures/Subgraph_Coupling.svg'.format(couple_name))
plt.show()
plt.close()

### Plot the average temporal correlation of subgraphs

In [ ]:
avg_expr_corr = subg_expr_corr.mean(axis=-1).mean(axis=-1)
avg_expr_corr[np.diag_indices_from(avg_expr_corr)] = 0
vmin=np.min(avg_expr_corr[np.nonzero(avg_expr_corr)])
vmax=np.max(avg_expr_corr[np.nonzero(avg_expr_corr)])

plt.figure()
ax = plt.subplot(111)
mat = ax.matshow(avg_expr_corr, cmap='plasma',
                 vmin=vmin, vmax=vmax)
plt.colorbar(mat, ax=ax)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
ax.set_xlabel('Subgraphs')
ax.set_ylabel('Subgraphs')

plt.savefig('./e04c-Figures/Temporal_Corr_Subgraph_Expression.svg')
plt.show()
plt.close()

### Plot the temporal correlation between subgraph types

In [ ]:
### Reproducibility
for set_id, (inter_lvoo, intra_lvoo, betwn_lvoo) in enumerate(zip(expr_corr_inter_mod_dist_lvoo,
                                                                  expr_corr_intra_mod_dist_lvoo,
                                                                  expr_corr_betwn_mod_dist_lvoo)):
    print('\n --- Reproducibility: Set {} ---'.format(set_id))
    
    plt.figure()
    ax = plt.subplot(111)
    bplot = ax.boxplot([inter_lvoo,
                        betwn_lvoo,
                        intra_lvoo], patch_artist=True);
    Echobase.Plotting.fig_format.set_box_color(bplot, 'k', [[0.0, 0.375, 1.0],
                                                            [0.0, 0.375, 1.0],
                                                            [0.0, 0.375, 1.0]])
    # Axis Settings
    ax.set_ylim([-0.3, 0.3])
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    ax.set_ylabel('Temporal Correlation of Subgraph Expression');
    ax.set_xticklabels(['Amongst\nInter-Module\nSubgraphs', 
                        'Between\nInter- and Intra-Module\nSubgraphs',
                        'Amongst\nIntra-Module\nSubgraphs']);

    plt.savefig('./e04c-Figures/Temporal_Corr_Subgraph_Expression-Module_Type-LVOO_{}.svg'.format(set_id))
    plt.close()    
    
    print(stats.ttest_rel(inter_lvoo, betwn_lvoo))
    print(stats.ttest_rel(betwn_lvoo, intra_lvoo))
    print(stats.ttest_rel(inter_lvoo, intra_lvoo))

### Module Reorganization via Subgraph Expression Coupling

In [ ]:
# Load Temporal Expression Correlation
df = np.load('{}/Subgraph_Module_Dynamics.npz'.format(path_ExpData), mmap_mode='r')
module_flex = df['module_flex']

n_subj, n_run, n_node, n_seed = module_flex.shape

# Compute module flexibility for subjects
module_flex_dist_lvoo = []
module_flex_dist_lvoo.append(module_flex.mean(axis=-1).mean(axis=-1).mean(axis=-1))
for r_id in xrange(n_run):
    lvin = np.setdiff1d(range(n_run), r_id)
    module_flex_dist_lvoo.append(module_flex[:, lvin, :, :].mean(axis=-1).mean(axis=-1).mean(axis=-1))

### Reproducibility
for set_id, (flex_lvoo, inter_lvoo, intra_lvoo, betwn_lvoo) in enumerate(zip(module_flex_dist_lvoo,
                                                                             expr_corr_inter_mod_dist_lvoo,
                                                                             expr_corr_intra_mod_dist_lvoo,
                                                                             expr_corr_betwn_mod_dist_lvoo)):
    print('\n --- Reproducibility: Set {} ---'.format(set_id))
    
    ### Inter-Module Subgraph
    plt.figure()
    ax = plt.subplot(111)

    m, b, rho_inter, _, _ = stats.linregress(inter_lvoo,
                                             flex_lvoo)
    ax.scatter(inter_lvoo, flex_lvoo, 
               lw=0, s=30.0, color=[0.20, 0.20, 0.75])
    x = np.array([-0.05, 0.3])
    y = m * x + b
    ax.plot(x, y, color=[0.2, 0.2, 0.2])

    # Axis Settings
    ax.set_ylim([0.09, 0.14])
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    ax.set_xlabel('Temporal Correlation')
    ax.set_ylabel('Module Flexibility');

    plt.savefig('./e04c-Figures/Inter_Module_Expr_Corr-Module_Flex-LVOO_{}.svg'.format(set_id))
    plt.close()


    ### Intra-Module Subgraph
    plt.figure()
    ax = plt.subplot(111)

    m, b, rho_intra, _, _ = stats.linregress(intra_lvoo,
                                             flex_lvoo)
    ax.scatter(intra_lvoo, flex_lvoo, 
               lw=0, s=30.0, color=[0.75, 0.20, 0.20])
    x = np.array([-0.07, -0.02])
    y = m * x + b
    ax.plot(x, y, color=[0.2, 0.2, 0.2])

    # Axis Settings
    ax.set_ylim([0.09, 0.14])    
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    ax.set_xlabel('Temporal Correlation')
    ax.set_ylabel('Module Flexibility');

    plt.savefig('./e04c-Figures/Intra_Module_Expr_Corr-Module_Flex-LVOO_{}.svg'.format(set_id))
    plt.close()


    ### Inter/Intra-Module Subgraph
    plt.figure()
    ax = plt.subplot(111)

    m, b, rho_betwn, _, _ = stats.linregress(betwn_lvoo,
                                             flex_lvoo)
    ax.scatter(betwn_lvoo, flex_lvoo, 
               lw=0, s=30.0, color=[0.20, 0.20, 0.20])
    x = np.array([-0.05, 0.05])
    y = m * x + b
    ax.plot(x, y, color=[0.2, 0.2, 0.2])

    # Axis Settings
    ax.set_ylim([0.09, 0.14])    
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    ax.set_xlabel('Temporal Correlation')
    ax.set_ylabel('Module Flexibility');

    plt.savefig('./e04c-Figures/Betwn_Module_Expr_Corr-Module_Flex-LVOO_{}.svg'.format(set_id))
    plt.close()


    ### Correlation
    n_null = 100000
    rho_inter_null = []
    rho_intra_null = []
    rho_betwn_null = []
    for n_i in xrange(n_null):
        rho, pv = stats.pearsonr(np.random.permutation(flex_lvoo), np.random.permutation(inter_lvoo))
        rho_inter_null.append(rho)

        rho, pv = stats.pearsonr(np.random.permutation(flex_lvoo), np.random.permutation(intra_lvoo))
        rho_intra_null.append(rho)

        rho, pv = stats.pearsonr(np.random.permutation(flex_lvoo), np.random.permutation(betwn_lvoo))
        rho_betwn_null.append(rho)

    if rho_inter < 0:    
        pv_inter_null = np.mean(rho_inter_null < rho_inter)
    else:
        pv_inter_null = np.mean(rho_inter_null > rho_inter)        
    if rho_intra < 0:    
        pv_intra_null = np.mean(rho_intra_null < rho_intra)
    else:
        pv_intra_null = np.mean(rho_intra_null > rho_intra)
    if rho_betwn < 0:        
        pv_betwn_null = np.mean(rho_betwn_null < rho_betwn)
    else:
        pv_betwn_null = np.mean(rho_betwn_null > rho_betwn)    

    print('Inter-Module Subgraphs: rho={}, pv={}'.format(rho_inter, pv_inter_null))
    print('Intra-Module Subgraphs: rho={}, pv={}'.format(rho_intra, pv_intra_null))
    print('Between Inter/Intra-Module Subgraphs: rho={}, pv={}'.format(rho_betwn, pv_betwn_null))

## Regional variation in module flexibility and subgraph coupling
Prior work shows that individual brain regions have varying degree of flexibility in shifting between functional modules.

In [ ]:
# Load Subgraph Data
data_sub = np.load(data_dict['subgraph_path'], mmap_mode='r')
fac_subgraph = data_sub['fac_subnet']
subg_adj = np.array([convert_conn_vec_to_adj_matr(subg_conn) for subg_conn in fac_subgraph])
n_fac = fac_subgraph.shape[0]

# Load Module Flexibility and Temporal Expression Correlation
df = np.load('{}/Subgraph_Module_Dynamics.npz'.format(path_ExpData), mmap_mode='r')
module_flex = df['module_flex']
subg_expr_corr = df['subgraph_expr_corr']

n_subj, n_run, n_node, n_seed = module_flex.shape

# Compute regional flexibility
node_flex_lvoo = []
node_flex_lvoo.append(module_flex.mean(axis=-1).mean(axis=1))
for r_id in xrange(n_run):
    lvin = np.setdiff1d(range(n_run), r_id)
    node_flex_lvoo.append(module_flex[:, lvin, :, :].mean(axis=-1).mean(axis=1))

# Compute node coupling
ns_fac_subg = np.sum(subg_adj, axis=1)
avg_subg_expr_corr = subg_expr_corr.mean(axis=0).mean(axis=-1)  # Subgraph_ID x Subject_ID
node_couple_lvoo = []
node_couple_lvoo.append(np.dot(avg_subg_expr_corr.T, ns_fac_subg))
for r_id in xrange(n_run):
    lvin = np.setdiff1d(range(n_run), r_id)
    avg_subg_expr_corr = subg_expr_corr[:, :, :, lvin].mean(axis=0).mean(axis=-1)  # Subgraph_ID x Subject_ID
    node_couple_lvoo.append(np.dot(avg_subg_expr_corr.T, ns_fac_subg))

### Reproducibility
for set_id, (nd_flex_lvoo, nd_couple_lvoo) in enumerate(zip(node_flex_lvoo,
                                                            node_couple_lvoo)):
                                                            
    print('\n --- Reproducibility: Set {} ---'.format(set_id))
    
    plt.figure()
    ax = plt.subplot(111)
    m, b, rho, pval, _ = stats.linregress(nd_couple_lvoo.mean(axis=0),
                                          nd_flex_lvoo.mean(axis=0),)
    ax.scatter(nd_couple_lvoo.mean(axis=0),
               nd_flex_lvoo.mean(axis=0),
               lw=0, s=10.0, color=[0.20, 0.20, 0.20])
    x = np.array([0.6, 1.2])
    y = m * x + b
    ax.plot(x, y, color=[0.20, 0.20, 0.20])

    # Axis Settings
    #ax.set_ylim([0.04, 0.20])
    #ax.set_xlim([0.6, 1.2])
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    ax.set_xlabel('Node Coupling')
    ax.set_ylabel('Node Flexibility');

    print('Node Flexibility and Coupling: rho={}, pv={}'.format(rho, pval))

    plt.savefig('./e04c-Figures/Node_Flex-Node_Couple-LVOO_{}.svg'.format(set_id))
    plt.close()

### Project Module-based node flexibility onto Brain

In [ ]:
import nibabel as nib

# Average over subjects
node_flex = node_flex_lvoo[0].mean(axis=0)
n_node = node_flex.shape[0]
node_flex = (node_flex - node_flex.mean()) / node_flex.std()

# Set the colormap
vmin = -2
vmax = 2
cm_sm = plt.cm.ScalarMappable()
cm_sm.set_cmap('plasma')
cm_sm.set_clim(vmin=vmin, vmax=vmax)
rgb = 255*cm_sm.to_rgba(node_flex)

# Read the ROI CSV
roi_csv = pd.read_csv('{}/Atlas/HOA112_Labels.csv'.format(path_CoreData))

# Map colors to ROI labels
for hemi in ['L', 'R']:
    f = open('{}/node_flex-hemi-{}.txt'.format(path_ExpData, hemi), 'w')
    f.write('{:3d}    {:3d} {:3d} {:3d}    {:1d} {:1d} {:1d}    "{}"\n'.format(0,0,0,0,0,0,0,'Clear Label'))
    
    for n_i in xrange(n_node):
        if roi_csv['Hemisphere'][n_i] == hemi:
            f.write('{:3d}    {:3d} {:3d} {:3d}    {:1d} {:1d} {:1d}    "{}"\n'.format(n_i+1,
                                                                                     int(rgb[n_i,0]), int(rgb[n_i,1]), int(rgb[n_i,2]),
                                                                                     1, 1, 1, roi_csv['Nomenclature'][n_i]))
        else:
            f.write('{:3d}    {:3d} {:3d} {:3d}    {:1d} {:1d} {:1d}    "{}"\n'.format(n_i+1,
                                                                                     int(rgb[n_i,0]), int(rgb[n_i,1]), int(rgb[n_i,2]),
                                                                                     0, 0, 0, roi_csv['Nomenclature'][n_i]))
    f.close()

### Project Subgraph-based node coupling onto Brain

In [ ]:
import nibabel as nib

# Average over subjects
node_couple = node_couple_lvoo[0].mean(axis=0)
n_node = node_couple.shape[0]
node_couple = (node_couple - node_couple.mean()) / node_couple.std()

# Set the colormap
vmin = -2
vmax = 2
cm_sm = plt.cm.ScalarMappable()
cm_sm.set_cmap('plasma')
cm_sm.set_clim(vmin=vmin, vmax=vmax)
rgb = 255*cm_sm.to_rgba(node_couple)

# Read the ROI CSV
roi_csv = pd.read_csv('{}/Atlas/HOA112_Labels.csv'.format(path_CoreData))

# Map colors to ROI labels
for hemi in ['L', 'R']:
    f = open('{}/node_couple-hemi-{}.txt'.format(path_ExpData, hemi), 'w')
    f.write('{:3d}    {:3d} {:3d} {:3d}    {:1d} {:1d} {:1d}    "{}"\n'.format(0,0,0,0,0,0,0,'Clear Label'))
    
    for n_i in xrange(n_node):
        if roi_csv['Hemisphere'][n_i] == hemi:
            f.write('{:3d}    {:3d} {:3d} {:3d}    {:1d} {:1d} {:1d}    "{}"\n'.format(n_i+1,
                                                                                     int(rgb[n_i,0]), int(rgb[n_i,1]), int(rgb[n_i,2]),
                                                                                     1, 1, 1, roi_csv['Nomenclature'][n_i]))
        else:
            f.write('{:3d}    {:3d} {:3d} {:3d}    {:1d} {:1d} {:1d}    "{}"\n'.format(n_i+1,
                                                                                     int(rgb[n_i,0]), int(rgb[n_i,1]), int(rgb[n_i,2]),
                                                                                     0, 0, 0, roi_csv['Nomenclature'][n_i]))
    f.close()